## Some State Space Model Tricks 

In this notebook, I plan to explore a few tricks that can be easily implemented for Bayesian estimation of state space models when using the joint likelihood approach described in ["Differentiable State-Space Models
and Hamiltonian Monte Carlo Estimation"](https://donskerclass.github.io/files/pdf/hmc_dssm.pdf) and implemented in the accompanying library [DifferentiableStateSpaceModels.jl](https://github.com/HighDimensionalEconLab/DifferentiableStateSpaceModels.jl). For an introduction to use of that library to fit linear and nonlinear versions of a simple model (a Real Business Cycle model), see the [tutorial notebok](https://nbviewer.org/github/HighDimensionalEconLab/DifferentiableStateSpaceModels.jl/blob/main/notebooks/estimate_rbc.ipynb). 

This notebook will build on that one (with much of the code copied), to describe how to introduce two easy-to-add but empirically very helpful features

1. Non-Gaussian (Student t) innovations, to better describe the heavy-tailed nature of economic series and measurements.
2. Stochastic volatility, to match observed heteroskedasticity in economic data.

These will be introduced using the modeling approach of [Chib, Shin, and Tan (2021)](https://docs.google.com/a/slu.edu/viewer?a=v&pid=sites&srcid=c2x1LmVkdXx0YW5mfGd4OmU1MTI5NTNmNTc2ZmRlOQ) in the context of a linearized model, for which, due to certainty equivalence, the error process and the equilibrium computation can be kept separate. This allows us to separate the DSGE solver component of the problem, implemented in `DifferentiableStateSpaceModels.jl`, from the Bayesian probability model setup, which can be done in a probabilistic programming environment, here that of [Turing.jl](https://turing.ml/). 

The reason this becomes simple in such a language is that it separates modeling from inference, so replacing a distribution in the model just requires changing the model setup. The reason this doesn't cause any issues in the inference is that when using the joint likelihood approach to state space models (that is, just treating the latent states the same as any other parameters), the sampling algorithm doesn't rely on any features of the distribution like (conditional) Gaussianity. The NUTS sampler does need derivatives, however, which creates some implementation issues, at least for the stochastic volatility part. Fortunately these are pretty easy to get around with some coding style changes.  

My goal in this is not a formal comparison of samplers (I'm sure their custom sampling algorithm is great), just an illustration of a set of things that can be done in a probabilistic programming language even without deriving new methods.

In [1]:
import Pkg; Pkg.instantiate();

In [2]:
import Pkg; Pkg.add("DifferentiableStateSpaceModels")

    Updating registry at `C:\Users\fm007\.julia\registries\General.toml`


   Resolving package versions...

  No Changes to `C:\Users\fm007\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\fm007\.julia\environments\v1.8\Manifest.toml`


In [3]:
## Pkg.add.(["DifferentiableStateSpaceModels" , "DifferenceEquations", "LinearAlgebra", "Zygote", "Distributions", "DiffEqBase", "Symbolics", "Plots", "Random", "StatsPlots","BenchmarkTools","LaTeXStrings"])


    Updating registry at `C:\Users\fm007\.julia\registries\General.toml`
   Resolving package versions...


    Updating `C:\Users\fm007\.julia\environments\v1.8\Project.toml`
  [b964fa9f] + LaTeXStrings v1.3.0
  No Changes to `C:\Users\fm007\.julia\environments\v1.8\Manifest.toml`


In [1]:
using DifferentiableStateSpaceModels, DifferenceEquations, LinearAlgebra, Zygote, Distributions, DiffEqBase, Symbolics, Plots, Random, StatsPlots

In [6]:
#Build the RBC model
∞ = Inf
@variables α, β, ρ, δ, σ, Ω_1
@variables t::Integer, k(..), z(..), c(..), q(..)

x = [k, z] # states
y = [c, q] # controls
p = [α, β, ρ, δ, σ, Ω_1] # parameters

H = [1 / c(t) - (β / c(t + 1)) * (α * exp(z(t + 1)) * k(t + 1)^(α - 1) + (1 - δ)),
     c(t) + k(t + 1) - (1 - δ) * k(t) - q(t),
     q(t) - exp(z(t)) * k(t)^α,
     z(t + 1) - ρ * z(t)]  # system of model equations

# analytic solutions for the steady state.  Could pass initial values and run solver and use initial values with steady_states_iv
steady_states = [k(∞) ~ (((1 / β) - 1 + δ) / α)^(1 / (α - 1)),
                 z(∞) ~ 0,
                 c(∞) ~ (((1 / β) - 1 + δ) / α)^(α / (α - 1)) -
                        δ * (((1 / β) - 1 + δ) / α)^(1 / (α - 1)),
                 q(∞) ~ (((1 / β) - 1 + δ) / α)^(α / (α - 1))]


Γ = [σ;;] # matrix for the 1 shock.  The [;;] notation just makes it a matrix rather than vector in julia
η = [0; -1;;] # η is n_x * n_ϵ matrix.  The [;;] notation just makes it a matrix rather than vector in julia

# observation matrix.  order is "y" then "x" variables, so [c,q,k,z] in this example
Q = [1.0 0  0   0; # select c as first "z" observable
     0   0  1.0 0] # select k as second "z" observable

# diagonal cholesky of covariance matrix for observation noise (so these are standard deviations).  Non-diagonal observation noise not currently supported
Ω = [Ω_1, Ω_1]

# Generates the files and includes if required.  If the model is already created, then just loads
overwrite_model_cache  = true
model_rbc = @make_and_include_perturbation_model("rbc_notebook_example", H, (; t, y, x, p, steady_states, Γ, Ω, η, Q, overwrite_model_cache)) # Convenience macro.  Saves as ".function_cache/rbc_notebook_example.jl"

UndefVarError: UndefVarError: max_order not defined

We use as our basic model an RBC with 1 shock (productivity) and 2 observables (consumption and capital) taken from the [tutorial notebok](https://github.com/HighDimensionalEconLab/DifferentiableStateSpaceModels.jl/blob/main/notebooks/estimate_rbc.ipynb). Here's a reminder of the model equations:

In [6]:
model_H_latex(model_rbc)

UndefVarError: UndefVarError: model_rbc not defined

Where we change from the tutorial version is that instead of using Gaussian shocks, which are traditional, we change their distribution. I'm going to start with the simplest modification, which is to change the shocks to be Student-t distributed. If this all works I will attempt to also add stochastic volatility (which requires adding a loop). I start by simulating some data:

In [7]:
#Solve model at some fixed parameters
p_f = (ρ = 0.2, δ = 0.02, σ = 0.01, Ω_1 = 0.01) # Fixed parameters
p_d = (α = 0.5, β = 0.95) # Pseudo-true values
m = model_rbc  # ensure notebook executed above
sol = generate_perturbation(m, p_d, p_f) # Solution to the first-order RBC

UndefVarError: UndefVarError: model_rbc not defined

In [8]:
# Simulate T observations from a random initial condition
T = 20
Random.seed!(12345) #Fix seed to reproduce data
dof = 4 #Student t degrees of freedom
shockdist = TDist(dof) #Shocks are student-t

# draw from t scaled by approximate invariant variance) for the initial condition
x_iv = sol.x_ergodic_var * rand(shockdist,sol.n_x)

UndefVarError: UndefVarError: Random not defined

In [9]:
# Generate noise sequence
noiseshocks = rand(shockdist,T)
noise = Matrix(noiseshocks') # the ϵ shocks are "noise" in DifferenceEquations for SciML compatibility 

UndefVarError: UndefVarError: shockdist not defined

In [10]:
#Solve problem forward with Student-t noise
problem = LinearStateSpaceProblem(sol, x_iv, (0, T); noise)
sim=solve(problem)
# Collapse to simulated observables as a matrix  - as required by current DifferenceEquations.jl likelihood
# see https://github.com/SciML/DifferenceEquations.jl/issues/55 for direct support of this datastructure
z_rbc = hcat(sim.z...)
plot(sim)

UndefVarError: UndefVarError: noise not defined

That looks like a nice heavy tailed simulation: we see some big swings that might create problems for a Gaussian likelihood. To estimate, build the model in Turing, identical to the benchmark model in the tutorial notebook with Gaussian noise when implemented with the joint approach, but replacing it with Student-t.

In [11]:
using Turing
using Turing: @addlogprob!
Turing.setadbackend(:zygote);  # Especially when we sample the latent noise, we will require high-dimensional gradients with reverse-mode AD

MethodError: MethodError: no method matching setadbackend(::Val{:zygote})
Closest candidates are:
  setadbackend(!Matched::Symbol) at C:\Users\fm007\.julia\packages\AdvancedVI\hVQ2g\src\ad.jl:5
  setadbackend(!Matched::Val{:forward_diff}) at C:\Users\fm007\.julia\packages\AdvancedVI\hVQ2g\src\ad.jl:6
  setadbackend(!Matched::Val{:forwarddiff}) at C:\Users\fm007\.julia\packages\AdvancedVI\hVQ2g\src\ad.jl:10
  ...

In [12]:
# Turing model definition
@model function rbc_1_t_joint(z, m, p_f, dof, cache, settings)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (; α, β)
    T = size(z, 2)
    xnought ~ filldist(TDist(dof),m.n_x) #Initial shocks 
    ϵ_draw ~ filldist(TDist(dof),m.n_ϵ * T) #Shocks are t-distributed!
    ϵ = reshape(ϵ_draw, m.n_ϵ, T)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache, settings) 
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    x_iv = sol.x_ergodic_var * xnought #scale initial condition to ergodic variance
    problem = LinearStateSpaceProblem(sol, x_iv, (0, T), observables = z, noise=ϵ)
    @addlogprob! solve(problem, DirectIteration()).logpdf # should choose DirectIteration() by default if not provided
end
cache = SolverCache(model_rbc, Val(1),  [:α, :β])
settings = PerturbationSolverSettings(; print_level = 0)
p_f = (ρ = 0.2, δ = 0.02, σ = 0.01, Ω_1 = 0.01) # Fixed parameters
z = z_rbc # simulated in previous steps
turing_model = rbc_1_t_joint(z, model_rbc, p_f, dof, cache, settings) # passing observables from before 

n_samples = 300
n_adapts = 50
δ = 0.65
alg = NUTS(n_adapts,δ)
chain_1_joint = sample(turing_model, alg, n_samples; progress = true)

UndefVarError: UndefVarError: model_rbc not defined

In [13]:
#Plot the chains and posteriors
plot(chain_1_joint[["α"]]; colordim=:parameter, legend=true)

UndefVarError: UndefVarError: chain_1_joint not defined

In [14]:
plot(chain_1_joint[["β"]]; colordim=:parameter, legend=true)

UndefVarError: UndefVarError: chain_1_joint not defined

In [15]:
#Plot true and estimated latents to see how well we backed them out
symbol_to_int(s) = parse(Int, string(s)[9:end-1])
ϵ_chain = sort(chain_1_joint[:, [Symbol("ϵ_draw[$a]") for a in 1:21], 1], lt = (x,y) -> symbol_to_int(x) < symbol_to_int(y))
tmp = describe(ϵ_chain)
ϵ_mean = tmp[1][:, 2]
ϵ_std = tmp[1][:, 3]
plot(ϵ_mean[2:end], ribbon=2 * ϵ_std[2:end], label="Posterior mean", title = "First-Order Joint: Estimated Latents")
plot!(noise', label="True values")

UndefVarError: UndefVarError: chain_1_joint not defined

That's really pretty good! From the table and plots we also did a decent job with $\alpha$ and $\beta$, for such a short MCMC chain.

So, that was changing to student-t shocks for the latent states. Once we had the data simulated, it required only **one change in the code** in the Turing model, from `MvNormal()` for multivariate normal to `filldist(Tdist())` for a collection of Student-t's. No use of fun special properties of the distribution like the scale-mixture-of-normals representation or fancy Gibbs samplers with closed form marginals were required (nor, thank heavens, anything like a particle filter), just sampling the latent states as if they were parameters. I make no guarantees that this approach is fast, but with HMC it isn't bad at all, and it is very easy, and as a bonus can be carried out with whatever distribution you want, not just well-behaved ones. So go ahead and start fitting your favorite residual error distributions!

### Next step: stochastic volatility

Stochastic volatility should be feasible with similar methods, but requires a bit more of a change from the existing code. The issue here is `LinearStateSpaceProblem` in `DifferenceEquations.jl` is currently set up only for Linear Time Invariant systems, and this extension makes it time varying. Fortunately, iteration is just a for loop, so this will be a nice exercise in demonstrating how to write a function compatible with the reverse mode automatic differentiation system in Zygote.

In [16]:
# Simulate T observations from a random initial condition
T = 50
Random.seed!(12435) #Fix seed to reproduce data
dof = 4 #Student t degrees of freedom
shockdist = TDist(dof) #Shocks are student-t
ρ_σ = 0.5 #Persistence of log volatility
μ_σ = 1. #Mean of (prescaling) volatility
σ_σ = 0.1 #Volatility of volatility

# draw from t scaled by approximate invariant variance) for the initial condition
x_iv = sol.x_ergodic_var * rand(shockdist,sol.n_x)

# Generate noise sequence
noise = Matrix(rand(shockdist,T)') # the ϵ shocks are "noise"
volshocks = Matrix(rand(MvNormal(T,1.0))') # the volatility shocks are log-normal
obsshocks = reshape(rand(MvNormal(T*sol.n_z,p_f[:Ω_1])), sol.n_z, T) #Gaussian observation noise

UndefVarError: UndefVarError: Random not defined

The following code is an extension of the code for simulating data from linear state space models in `DifferenceEquations.jl`, extended to add a stochastic volatility element of the form
$$vol_t = \rho_\sigma * vol_{t-1} + (1 - \rho_\sigma) * \mu_\sigma + \sigma_\sigma * \zeta_{t-1}$$
where $exp(vol_t)$ is the standard deviation of the shock at time $t$, and $\zeta_{t}$ are i.i.d. normal shocks. (We could make them non-normal, but this is fine for now).

The one thing to note about the way the code is implemented is that it uses exclusively in-place operations inside the loop, which means that it fixes the sequence once then updates it rather than allocating a new variable at each step in the loop. This is a useful programming trick for optimizing memory usage and speed, which is why we used it in the library: see https://book.sciml.ai/ for a wonderful course on writing super efficient Julia code. But it's kind of hard to read, and more importantly, would create problems down the road if we tried to use it directly inside our estimation without some additional tricks. So I'll rewrite it in a simpler way for that task. 

In [17]:
#Extract solution matrices
A = sol.A
B = sol.B
C = sol.C
D = sol.D

# Initialize
u = [zero(x_iv) for _ in 1:T]
u[1] .= x_iv
vol = [zeros(1) for _ in 1:T]
vol[1] = [μ_σ] #Start at mean: could make random but won't for now
#Allocate sequence
z = [zeros(size(C, 1)) for _ in 1:T] 
mul!(z[1], C, u[1])  # update the first of z
for t in 2:T
        mul!(u[t], A, u[t - 1]) # sets u[t] = A * u[t - 1]
        mul!(vol[t], ρ_σ, vol[t-1])
        vol[t] .+= (1 - ρ_σ) * μ_σ
        mul!(vol[t], σ_σ, view(volshocks, :, t - 1),1,1) # adds σ_σ * volshocks[t-1] to vol[t]
        mul!(u[t], exp(vol[t][]) .* B, view(noise, :, t - 1),1,1)
        mul!(z[t], C, u[t]) 
end
for t in 1:T #Add observation noise
        z[t] .+= view(obsshocks,:,t)
end

UndefVarError: UndefVarError: sol not defined

In [18]:
z_data = hcat(z...)
plot(z_data') #Plot k and z from simulation

UndefVarError: UndefVarError: z not defined

In [19]:
plot(hcat(vol...)') #Plot the latent volatility state

UndefVarError: UndefVarError: vol not defined

That worked fine for simulation, so the next step is to create the likelihood function corresponding to that simulation. A simple approach would just be take that code and replace the sampling components with density evaluations, which is essentially how our code in `DifferenceEquations.jl` works.
Unfortunately, I can't do that exactly because the AD framework, `Zygote`, [doesn't allow mutation](https://fluxml.ai/Zygote.jl/latest/limitations/). What this means is that, because it works by following your function along its execution trace, it needs to be able to have a unique address for each evaluation, which gets messed up if your functions work by reallocating the values in an existing object. In practice, this can be resolved either by functional programming or by hiding the non-functional parts away from the AD system.  
There are a few ways to do that in practice.

- One way is by hiding the mutation in an explicitly constructed adjoint function, creating a custom `rrule` in `ChainRules.jl`. This is feasible and can yield highly performant code, which is what we did for the (very similar) adjoint formulas in `DifferenceEquations.jl`. This is also a bit tedious and involves math (mostly [transposing matrices](https://juliadiff.org/ChainRulesCore.jl/dev/maths/arrays.html)), so I'm going to avoid it for now.
    
- Alternately, it ought to be doable by cleaning up the function to avoid the allocation parts. I think that will make things easier than the current code, which uses it for speed and memory reasons, not because of Zygote. Certainly getting rid of all those `mul!` statements ought to make the code easier to read. The only really tricky part about this is that to avoid preallocating the sequences, I will use a special data structure called Zygote.Buffer() which stores the values but isn't touched by the AD, and wrap the code containing the buffer in a function. This is maybe not so performant, but it's easy.
    
*Note to applied researchers for whom the above discussion sounded scary* (eg, me before I spent a hunk of time learning about this stuff): for most common models and operations the AD systems are fully established and you will never have to build these constructs yourself. And if you do end up needing to build custom adjoints because of some gnarly custom modeling code, tools are getting easier to use: the Python-based AD system JAX has a nice feature called [automatic transposition](https://jax.readthedocs.io/en/latest/notebooks/Custom_derivative_rules_for_Python_code.html) where all you have to do is write a function that computes the derivative of your function of interest.

In [20]:
#Likelihood evaluation function using `Zygote.Buffer()` to create internal arrays that don't interfere with gradients.
function svlikelihood2(A,B,C,D,x_iv,Ω_1,μ_σ,ρ_σ,σ_σ,observables,noise,volshocks) #Accumulate likelihood
    # Initialize
    T = size(observables,2)
    u = Zygote.Buffer([zero(x_iv) for _ in 1:T]) #Fix type: Array of vector of vectors?
    vol = Zygote.Buffer([zeros(1) for _ in 1:T]) #Fix type: Array of vector of vectors?
    u[1] = x_iv 
    vol[1] = [μ_σ] #Start at mean: could make random but won't for now
    for t in 2:T
        vol[t] = ρ_σ * vol[t-1] .+ (1 - ρ_σ) * μ_σ .+ σ_σ * volshocks[t - 1]
        u[t] = A * u[t - 1] .+ exp.(vol[t]) .* (B * noise[t - 1])[:]
    end
    loglik = sum([logpdf(MvNormal(Diagonal(Ω_1 * ones(size(C, 1)))), observables[t] .- C * u[t]) for t in 1:T])
    return loglik
end

ll = svlikelihood2(sol.A,sol.B,sol.C,sol.D,x_iv,p_f[:Ω_1],μ_σ,ρ_σ,σ_σ,z_data,noise,volshocks)

gradient(x_iv->svlikelihood2(sol.A,sol.B,sol.C,sol.D,x_iv,p_f[:Ω_1],μ_σ,ρ_σ,σ_σ,z_data,noise,volshocks),[0., 0.])

UndefVarError: UndefVarError: sol not defined

As the gradient code demonstrates, automatic differentiation works for this version of the likelihood function.  Now let's stick it in our model.

In [21]:
# Turing model definition
@model function rbc_1_svt_jointseq(z, m, p_f, dof, cache, settings)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    ρ_σ ~ Beta(2.625, 2.625) #Persistence of log volatility
    μ_σ ~ Normal(1., 0.5) #Mean of (prescaling) volatility
    σ_σ ~ Uniform(0.03, 0.3) #Volatility of volatility
    p_d = (; α, β)
    T = size(z, 2)
    xnought ~ filldist(TDist(dof),m.n_x) #Initial shocks 
    ϵ_draw ~ filldist(TDist(dof),m.n_ϵ * T) #Shocks are t-distributed!
    ϵ = reshape(ϵ_draw, m.n_ϵ, T)
    vsdraw ~ MvNormal(T, 1.0)
    volshocks = reshape(vsdraw,1,T)   
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache, settings) 
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    x_iv = sol.x_ergodic_var * xnought #scale initial condition to ergodic variance
    @addlogprob! svlikelihood2(sol.A,sol.B,sol.C,sol.D,x_iv,p_f[:Ω_1],μ_σ,ρ_σ,σ_σ,z,ϵ,volshocks)
end

rbc_1_svt_jointseq (generic function with 2 methods)

In [22]:
cache = SolverCache(model_rbc, Val(1),  [:α, :β])
settings = PerturbationSolverSettings(; print_level = 0)
p_f = (ρ = 0.2, δ = 0.02, σ = 0.01, Ω_1 = 0.01) # Fixed parameters
z = z_data # simulated in previous steps
turing_model2 = rbc_1_svt_jointseq(z, model_rbc, p_f, dof, cache, settings) # passing observables from before 

n_samples = 1000
n_adapts = 100
δ = 0.65
alg = NUTS(n_adapts,δ)
chain_2_joint = sample(turing_model2, alg, n_samples; progress = true)

UndefVarError: UndefVarError: model_rbc not defined

In [23]:
plot(chain_2_joint[["α"]]; colordim=:parameter, legend=true)

UndefVarError: UndefVarError: chain_2_joint not defined

In [24]:
plot(chain_2_joint[["β"]]; colordim=:parameter, legend=true)

UndefVarError: UndefVarError: chain_2_joint not defined

In [25]:
#Plot true and estimated latents to see how well we backed them out
symbol_to_int(s) = parse(Int, string(s)[9:end-1])
ϵ_chain = sort(chain_2_joint[:, [Symbol("ϵ_draw[$a]") for a in 1:50], 1], lt = (x,y) -> symbol_to_int(x) < symbol_to_int(y))
tmp = describe(ϵ_chain)
ϵ_mean = tmp[1][:, 2]
ϵ_std = tmp[1][:, 3]
plot(ϵ_mean[1:end], ribbon=2 * ϵ_std[1:end], label="Posterior mean", title = "First-Order Joint: Estimated Shocks")
plot!(noise', label="True values")

UndefVarError: UndefVarError: chain_2_joint not defined

In [26]:
#Plot true and estimated volatility shocks to see how well we backed them out
symbol_to_int(s) = parse(Int, string(s)[8:end-1])
v_chain = sort(chain_2_joint[:, [Symbol("vsdraw[$a]") for a in 1:50], 1], lt = (x,y) -> symbol_to_int(x) < symbol_to_int(y))
tmp = describe(v_chain)
v_mean = tmp[1][:, 2]
v_std = tmp[1][:, 3]
plot(v_mean[1:end], ribbon=2 * v_std[1:end], label="Posterior mean", title = "First-Order Joint: Estimated Volatility Shocks")
plot!(volshocks', label="True values")

UndefVarError: UndefVarError: chain_2_joint not defined

That did it! With a little bit of fiddling with data structures to handle some technical issues with automatic differentiation, we built a working probability model incorporating economic structure, non-Gaussian errors, and stochastic volatility. There are plenty of code optimizations that could be done if we wanted to make this faster (see https://book.sciml.ai/ for a comprehensive guide to writing fast Julia), but this gave us a simple working version to start from, that performs okay. The estimation accuracy isn't great here, but I suspect that's a feature of the model rather than the procedure. Specifically, there's just not a lot of info in the data points to back out the time-varying volatilities, so the posterior ends up quite wide and the rest of the parameters are correspondingly harder to estimate. I may consider tighter priors, longer data, or other features improving identification, for future runs.

In terms of economic content, this kind of model is likely to be extremely relevant for studying data post-pandemic, with huge outliers best matched by heavy-tailed distributions, and for dealing with questions where changes in volatility help identify responses ([e.g.](https://itskhoki.com/papers/Mussa.pdf)). Moreover, nothing relied explicitly on the form of the distribution or the volatility shocks, so you could easily incorporate features like skewed shock distributions or even stochastic skewness. Of course, the certainty equivalent case is somewhat limiting here, but you can capture the first order effects of volatility using higher order perturbation solution, which we provide, and, in principle, extend further to more general nonlinear solvers.  